# Assignment 5

Due: October 14, 2024

Name: Eric Wu

In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

sys.path.append('/content/drive/MyDrive/Colab Notebooks/CDS411')

Mounted at /content/drive


In [2]:
import bacon, floyd
import movies3 as mov
f1 = '/content/drive/MyDrive/Colab Notebooks/CDS411/movies1100.xlsx'
movies, actors, isin = mov.ReadData(f1)

## Problem 1
Write a Python script that returns the _aid_ (actor id) value for Kim Crosby?

In [ ]:
# your code and answer
'''print(actors.shape)
print(actors[:10,:])'''

aid_kc = mov.AidFromName(actors,'Kim','Crosby') #Returns actor Kim Crosby's aid
print(aid_kc)

376


## Problem 2
Write a Python script that returns the _mid_ (movie id) values for the movies with Amy Irving?

In [ ]:
# your code and answer
aid_ai = mov.AidFromName(actors,'Amy','Irving') #Returns actor Amy Irving's aid

aid_ai2mids = mov.MidsFromAid(isin, aid_ai) #Returns mids from Amy's aid

print(aid_ai2mids)

[242   6 243 994]


## Problem 3
Write a Python script that returns the titles of the movies Lena Horne is in.

In [ ]:
# your code and answer

aid_lh = mov.AidFromName(actors,'Lena','Horne') #Returns Lena Horne's aid

aid_lh2mids = mov.MidsFromAid(isin, aid_lh) #Finds mids for movies from Lena's aid

aid_lh2mids2titles = mov.TitlesFromMids(movies,aid_lh2mids) #Gets the titles from the mids that Lena has worked on

print(aid_lh2mids2titles)

['Stormy Weather', 'Cabin in the Sky', 'The Wiz']


## Problem 4
Write a Python script that returns the names of the actors that are in the movie named _Inherit the Wind_ .

In [ ]:
# your code and answer
mid_itw = mov.MidFromTitle(movies, 'Inherit the Wind') #Finds the mid for the movie

mid_itw2aids = mov.AidsFromMid(isin,mid_itw) #Gets the aids that are associated with the mid

mid_itw2aids2actors = mov.NamesFromAids(actors,mid_itw2aids) #Gets the names associated with the aids associated with the movie

print(mid_itw2aids2actors)

[('Spencer', 'Tracy'), ('Gene', 'Kelly'), ('Harry', 'Morgan'), ('Norman', 'Fell')]


## Problem 5
What are the _aid_ values in the shortest path from Spencer Tracy to Lena Horne?

In [6]:
import numpy as np

def MakeG( isin ):
  '''
  Fixed func MakeG as the MakeG in bacon.py bases the size of G on the entries of isin, which does not take in account all of the aids in set(aid)
  input: isin df
  return: initialized G matrix
  '''
  ## list of unique aids + sort
  mat = np.array( isin )
  t = mat[:,2]+0
  aids = np.array(list(set(t)))
  ## allocate G
  #N = len(aids)
  N = 856 #It seems only 854 actors are actually connected, but the connected actors' aids go up to 856
  G = np.zeros((N+1,N+1))
  ## for each actor
  for i in aids:
    ## get mids
    mids = mov.MidsFromAid(isin, i)
    ## aids from actor's movies
    aids2 = mov.AidsFromMids( isin, mids )
    # populate G
    if i==771:
      print(aids2)
    for a in aids2:
        G[i-1, a-1] = 1
    ## return G
  print(G[770,770])
  return G

In [7]:
# your code and answer
G = MakeG(isin) #Initialize G
G,P = bacon.RunFloyd(G) #Fill in and correct G and create P

[640, 121, 770, 771, 334, 849, 182, 729, 221]
1.0
0 50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 

In [8]:
def Fpath(name1,name2,distance=True,find=True,actrs=actors,p=P,g=G):
  '''
  Finds the path from actor1 to actor2 and returns path
  input: Actor1 (string), Actor2 (string), Distance flag for printing distance between actor1 and actor2, find flag for printing the path, pass in actors df, P matx, and G matx
  return: path (list) between actor1 and actor2
  '''
  f1,l1 = name1.split()
  f2,l2 = name2.split()
  aid1 = mov.AidFromName(actors,f1,l1)
  aid2 = mov.AidFromName(actors,f2,l2)
  v,h = aid1-1,aid2-1

  if distance:
    print(f"Distance is {G[v,h]}")

  path = floyd.FindPath(P,v,h)
  if find:
    print(f'Path {path}')

  return path

In [ ]:
aids = np.array(Fpath('Spencer Tracy','Lena Horne',True,False))+1 #Get path of aids between actors
print(aids)

#print(mov.NameFromAid(actors,363))

Distance is 3.0
[363 364 440 827]


## Problem 6
What are the names of the movies that are in the shortest path from Aidan Quinn to Anthony Quinn

In [ ]:
def MoviesOfTwoActors(actors,isin,aid1, aid2):
  '''
  Gets the movies shared between two actors in form of mid
  inputs: actors df, isin df, aid1, aid2
  output: mids of the shared movies between actors
  '''

  mids1 = mov.MidsFromAid(isin,aid1)
  mids2 = mov.MidsFromAid(isin,aid2)
  mids = mov.CombineLists(mids1,mids2)
  return mids

In [ ]:
# your code and answer
aids = np.array(Fpath('Aidan Quinn','Anthony Quinn',True,False))+1 #Gets the path between the two actors in form of aid
#print(aids)

mids = list() #Initialize mid list
for i in range(3):
  mids.append(MoviesOfTwoActors(actors,isin,aids[i],aids[i+1])) #Loop N-1, appends the mid shared between two actors to mids list

print(mov.TitlesFromMids(movies,mids)) #Print out titles from the list of mids

Distance is 3.0
['Stakeout', 'The Big Fix', 'Last Action Hero']


## Problem 7
What is the longest path for Kevin Bacon?  Include actor names and movie titles grouped so that we know which actors are in which movies.

In [ ]:
# your code and answer
Gdist = (G<99999)*G #Filter out non connections
aid_kb = mov.AidFromName(actors,'Kevin','Bacon') #Gets Kevin Bacon
mx = Gdist[aid_kb-1,:].max() #Gets longest path associated with aid_kb
ndx = (Gdist[aid_kb-1,:] == mx).nonzero() #Gets index of mx
h = ndx[0]+1 #Gets h (aid is n+1)

print(aid_kb); print(mov.NameFromAid(actors,h)) #Gets name associated with aid, h
aids = np.array(Fpath('Kevin Bacon','Kim Crosby',True,False))+1#; print(aids) #Gets path between two actors

mids = list() #Initialize mids list
for i in range(5):
  mids.append(MoviesOfTwoActors(actors,isin, aids[i],aids[i+1])) #Loops to find shared mid between the two actor's aids

print(mids)
mids[2] = [176] #re-add second shared mid

#print('aids',aids, '\nmids', mids)

143
('Kim', 'Crosby')
Distance is 5.0
[[339], [94], [176, 177], [1036], [254]]


In [ ]:
names_a = mov.NamesFromAids(actors,aids)#; print(names_a) #Gets actor names
names_m = mov.TitlesFromMids(movies,mids); names_m[2] = [names_m[2], mov.TitleFromMid(movies,177)]#; print(names_m) #Gets movie names

grouped = list() #Initialize grouped list
for i in range(5):
  grouped.append([names_a[i],names_m[i]]) #Group actor and movie name associated with each "step"
print(grouped)

[[('Kevin', 'Bacon'), 'Planes, Trains and Automobiles'], [('Steve', 'Martin'), 'The Pink Panther 2'], [('John', 'Cleese'), ['The World is Not Enough', 'Die Another Day']], [('Pierce', 'Brosnan'), 'The Mirror Crack’d'], [('Tony', 'Curtis'), 'Tarzan in Manhattan']]


## Problem 8
Mentioned in the lectures is that most actors are connected.  Thus, there is one big collection of connected actors, and a few that are not.  Leonard DiCaprio is one of the actors in this big connection.  How many actors are in this group?

In [15]:
# your code and answer
import numpy as np

num_of_actrs = len(actors); print(f'The total number of actors is {num_of_actrs}')

The total number of actors is 856


In [22]:
aid_leo = mov.AidFromName(actors,'Leonardo','DiCaprio') # get leo's aid

print(f'The there are {sum(G[aid_leo-1,:]<=999999)} actors in the big web') # find all the actors connected to leo

The there are 837 actors in the big web
